# 한 사용자에게 개인 추천을 해주기
 - 보통 추천 시스템은 사용자에게 추천
 - 사용자에게 추천을 해주기 위해서 사용자 맞춤 협업 필터링 행렬 분해를 적용
 - 데이터는 kaggle의 https://www.kaggle.com/usernam3/shopify-app-store 를 전처리한 데이터입니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
import time
warnings.filterwarnings('ignore')
%matplotlib inline

- 사용자 평점이 있는 데이터

In [2]:
reviews = pd.read_csv('./dataset/reviews.csv',delimiter = None,encoding='utf-8')
user_rating = reviews[['app_id','author','rating']].dropna(axis=0)
print(user_rating.shape)
print(user_rating.info())
user_rating.head()

(447193, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 447193 entries, 0 to 447316
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app_id  447193 non-null  object
 1   author  447193 non-null  object
 2   rating  447193 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 13.6+ MB
None


,app_id,author,rating
0,b1da53a4-0474-4700-9620-bf386bc033fb,Consuela,5
1,b1da53a4-0474-4700-9620-bf386bc033fb,L'Atelier Global,5
2,b1da53a4-0474-4700-9620-bf386bc033fb,city'super E-Shop,5
3,b1da53a4-0474-4700-9620-bf386bc033fb,PortableHandwashing.com,5
4,b1da53a4-0474-4700-9620-bf386bc033fb,ICCTUNING,5


- 앱의 가중평점이 있는 데이터

In [3]:
app_df = pd.read_csv('./temp_data/app_df.csv',delimiter = None,encoding='utf-8')
app_rating = app_df[['app_id','wr_rating']].dropna(axis=0)
print(app_rating.shape)
print(app_rating.info())
app_rating.head()

(4750, 2)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4750 entries, 0 to 4749
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   app_id     4750 non-null   object 
 1   wr_rating  4750 non-null   float64
dtypes: float64(1), object(1)
memory usage: 111.3+ KB
None


,app_id,wr_rating
0,9e4748a9-7eda-4814-83b6-0537d44152b1,30.800631
1,d1476138-a608-4bb9-8d39-b30f3ca7617d,87.308207
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,43.588712
3,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,84.941515
4,7aac2a1f-ff03-4f38-aeb7-7619403a6f05,94.171158


- 사용자 별 영화평점 피벗 테이블 생성

In [4]:
%%time
user_rating_pt = user_rating.pivot_table(values='rating', index='author', columns='app_id').fillna(0)
user_rating_pt

Wall time: 2min 35s


app_id,00014403-ad17-4c9b-bbcf-5adc27ba18a9,000270a8-af0b-4862-82df-25decbcc8d86,0006d5af-c461-4bdb-b32e-a4fecd00846b,001ce7ab-14dc-45ce-98f1-1b42b61c86c1,00215fb3-ff7c-4111-8f0c-8e7e2a9027fb,005739cc-4d50-4013-ad4a-beca69e32037,007a1f24-093e-4de4-bfe8-d9b097f4cf3b,00ad6b55-3a26-42be-a185-ca3bf2a361dc,00b08a94-39ac-4a90-b7a4-d8a177d5d1e1,00be4dc9-4151-477f-aba4-c2ce97fd65d8,...,ff453576-4a68-48ec-8ffe-b1557fbf942b,ff684d29-6792-4fb0-b3e3-0112605a21be,ff6f258a-849f-4ff6-aab4-52c93b28458f,ff7b70fe-336a-4fbb-9556-0e7042e05aa9,ff8e0c0d-50b2-4c93-8c2f-1848b14dc142,ff9fcefb-cb60-4ff3-8ec8-75a6e89b9055,ffda2590-e98f-46b7-8608-ce242bfd0c07,ffda8a87-f35e-4d27-bee9-c3088a613e08,fff053c9-30de-426b-8a5f-3e393ff45ec0,fff4cec3-a140-45bd-b257-fec3759497a0
author,,,,,,,,,,,,,,,,,,,,,
"""Accountable Clothing""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""All Things Rugby""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""HOOKED ON LINGERIE""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""LIFT OFF"" by Andrew",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Raggy Stylz"" Doggy Fashion",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
💨Incensery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
🔥Fire Fashion & Decor🔥,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
🔥🔥🔥hot🌍sales🔥🔥🔥,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- matrix는 pivot_table 값을 numpy matrix로 만든 것

In [5]:
matrix = np.matrix(user_rating_pt)
matrix

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

- user_ratings_mean은 각 사용자 별 평균 평점

In [6]:
user_ratings_mean = np.mean(matrix, axis = 1)
user_ratings_mean

matrix([[0.00080364],
        [0.00214305],
        [0.00267881],
        ...,
        [0.00133941],
        [0.00267881],
        [0.00107152]])

- R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
- 데이터 정규화를 위함

In [7]:
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)
matrix_user_mean

matrix([[-0.00080364, -0.00080364, -0.00080364, ..., -0.00080364,
         -0.00080364, -0.00080364],
        [-0.00214305, -0.00214305, -0.00214305, ..., -0.00214305,
         -0.00214305, -0.00214305],
        [-0.00267881, -0.00267881, -0.00267881, ..., -0.00267881,
         -0.00267881, -0.00267881],
        ...,
        [-0.00133941, -0.00133941, -0.00133941, ..., -0.00133941,
         -0.00133941, -0.00133941],
        [-0.00267881, -0.00267881, -0.00267881, ..., -0.00267881,
         -0.00267881, -0.00267881],
        [-0.00107152, -0.00107152, -0.00107152, ..., -0.00107152,
         -0.00107152, -0.00107152]])

In [8]:
print(matrix.shape)
print(user_ratings_mean.shape)
print(matrix_user_mean.shape)

(299314, 3733)
(299314, 1)
(299314, 3733)


- matrix_user_mean의 결과를 보기 편하게 데이터 프레임으로 변환해 봄.

In [9]:
pd.DataFrame(matrix_user_mean, columns = user_rating_pt.columns, index=user_rating_pt.index)

app_id,00014403-ad17-4c9b-bbcf-5adc27ba18a9,000270a8-af0b-4862-82df-25decbcc8d86,0006d5af-c461-4bdb-b32e-a4fecd00846b,001ce7ab-14dc-45ce-98f1-1b42b61c86c1,00215fb3-ff7c-4111-8f0c-8e7e2a9027fb,005739cc-4d50-4013-ad4a-beca69e32037,007a1f24-093e-4de4-bfe8-d9b097f4cf3b,00ad6b55-3a26-42be-a185-ca3bf2a361dc,00b08a94-39ac-4a90-b7a4-d8a177d5d1e1,00be4dc9-4151-477f-aba4-c2ce97fd65d8,...,ff453576-4a68-48ec-8ffe-b1557fbf942b,ff684d29-6792-4fb0-b3e3-0112605a21be,ff6f258a-849f-4ff6-aab4-52c93b28458f,ff7b70fe-336a-4fbb-9556-0e7042e05aa9,ff8e0c0d-50b2-4c93-8c2f-1848b14dc142,ff9fcefb-cb60-4ff3-8ec8-75a6e89b9055,ffda2590-e98f-46b7-8608-ce242bfd0c07,ffda8a87-f35e-4d27-bee9-c3088a613e08,fff053c9-30de-426b-8a5f-3e393ff45ec0,fff4cec3-a140-45bd-b257-fec3759497a0
author,,,,,,,,,,,,,,,,,,,,,
"""Accountable Clothing""",-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,...,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804
"""All Things Rugby""",-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,...,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143,-0.002143
"""HOOKED ON LINGERIE""",-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,...,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679,-0.002679
"""LIFT OFF"" by Andrew",-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,...,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804,-0.000804
"""Raggy Stylz"" Doggy Fashion",-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,...,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965,-0.006965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
💨Incensery,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,...,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268,-0.000268
🔥Fire Fashion & Decor🔥,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,...,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339
🔥🔥🔥hot🌍sales🔥🔥🔥,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,...,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339,-0.001339


- 메모리 할당 문제로 연산오류가 떠서 데이터를 임의로 줄여줌

In [10]:
matrix_user_mean_s = matrix_user_mean[:int(len(matrix_user_mean)/5)]
matrix_user_mean_s.shape

(59862, 3733)

### scipy를 이용해 Truncated SVD를 구하기
 - scikit learn에서 제공해주는 TruncatedSVD는 U, Sigma, Vt 반환 값을 제공안함
 - 하지만, Scipy를 이용하면 이 반환값들을 제공받을 수 있음
 - 이렇게 사용하면 반환값이 U 행렬, Sigma 행렬, V 전치 행렬(Vt)가 나옴

In [11]:
from scipy.sparse.linalg import svds

In [12]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean_s, k = 12)

print(U.shape)
print(sigma.shape)
print(Vt.shape)

(59862, 12)
(12,)
(12, 3733)


- 현재 이 Sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태
- 0이 포함된 대칭행렬로 변환하기 위해 numpy의 diag를 이용

In [13]:
sigma__ = np.diag(sigma)
print(sigma__.shape)
print(sigma__[0])

(12, 12)
[139.1565232   0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.       ]


- U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다.
- 거기에 + 사용자 평균 rating을 적용한다.

In [14]:
# 행렬 계산을 맞춰주기 위함
user_ratings_mean_s =  user_ratings_mean[:int(len(user_ratings_mean)/5)]

svd_user_predicted_ratings = np.dot(np.dot(U, sigma__), Vt) + user_ratings_mean_s.reshape(-1, 1)
svd_user_predicted_ratings

matrix([[ 0.00079937,  0.00079787,  0.00079636, ...,  0.00079092,
          0.00080273,  0.00079847],
        [ 0.00195418,  0.00195207,  0.00212994, ...,  0.00234542,
          0.00194989,  0.00196639],
        [ 0.00244184,  0.00239196,  0.00244268, ...,  0.00245938,
          0.00254461,  0.00241942],
        ...,
        [-0.00054715, -0.00070624, -0.0004351 , ..., -0.00020557,
         -0.00025742, -0.00060141],
        [ 0.00125902,  0.00125047,  0.00125544, ...,  0.00131698,
          0.00127264,  0.00125625],
        [ 0.00102299,  0.00102485,  0.00104702, ...,  0.00103839,
          0.00101963,  0.00102424]])

In [15]:
user_rating_pt_s = user_rating_pt[:int(len(user_rating_pt)/5)]

df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = user_rating_pt.columns, index=user_rating_pt_s.index)
df_svd_preds

app_id,00014403-ad17-4c9b-bbcf-5adc27ba18a9,000270a8-af0b-4862-82df-25decbcc8d86,0006d5af-c461-4bdb-b32e-a4fecd00846b,001ce7ab-14dc-45ce-98f1-1b42b61c86c1,00215fb3-ff7c-4111-8f0c-8e7e2a9027fb,005739cc-4d50-4013-ad4a-beca69e32037,007a1f24-093e-4de4-bfe8-d9b097f4cf3b,00ad6b55-3a26-42be-a185-ca3bf2a361dc,00b08a94-39ac-4a90-b7a4-d8a177d5d1e1,00be4dc9-4151-477f-aba4-c2ce97fd65d8,...,ff453576-4a68-48ec-8ffe-b1557fbf942b,ff684d29-6792-4fb0-b3e3-0112605a21be,ff6f258a-849f-4ff6-aab4-52c93b28458f,ff7b70fe-336a-4fbb-9556-0e7042e05aa9,ff8e0c0d-50b2-4c93-8c2f-1848b14dc142,ff9fcefb-cb60-4ff3-8ec8-75a6e89b9055,ffda2590-e98f-46b7-8608-ce242bfd0c07,ffda8a87-f35e-4d27-bee9-c3088a613e08,fff053c9-30de-426b-8a5f-3e393ff45ec0,fff4cec3-a140-45bd-b257-fec3759497a0
author,,,,,,,,,,,,,,,,,,,,,
"""Accountable Clothing""",0.000799,0.000798,0.000796,0.000798,0.000798,0.000801,0.000798,0.000988,0.000787,0.000798,...,0.000798,0.000798,0.000799,0.000798,0.000798,0.000843,0.000798,0.000791,0.000803,0.000798
"""All Things Rugby""",0.001954,0.001952,0.002130,0.001952,0.001954,0.001964,0.001953,0.002220,0.002703,0.001952,...,0.001952,0.001952,0.001967,0.001952,0.001952,0.003996,0.001951,0.002345,0.001950,0.001966
"""HOOKED ON LINGERIE""",0.002442,0.002392,0.002443,0.002392,0.002399,0.002522,0.002402,0.009037,0.002616,0.002392,...,0.002392,0.002392,0.002443,0.002391,0.002392,0.005323,0.002402,0.002459,0.002545,0.002419
"""LIFT OFF"" by Andrew",0.000795,0.000795,0.000791,0.000795,0.000795,0.000793,0.000795,0.000698,0.000811,0.000795,...,0.000795,0.000795,0.000796,0.000795,0.000795,0.000843,0.000794,0.000841,0.000791,0.000797
"""Raggy Stylz"" Doggy Fashion",0.002452,0.002306,0.009146,0.002302,0.002389,0.001287,0.002359,-0.008802,0.006254,0.002311,...,0.002302,0.002306,0.003039,0.002298,0.002306,0.017978,0.002253,0.016836,0.001461,0.003029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Deallinois,0.001226,0.001227,0.001279,0.001227,0.001228,0.001215,0.001227,0.000656,0.001418,0.001227,...,0.001227,0.001227,0.001237,0.001227,0.001227,0.002209,0.001226,0.001365,0.001211,0.001236
Dealmagic,0.001136,0.001091,0.001111,0.001091,0.001097,0.001215,0.001100,0.006753,0.001174,0.001091,...,0.001091,0.001091,0.001138,0.001091,0.001091,0.003545,0.001100,0.001119,0.001233,0.001116
Dealmakertoday,-0.000547,-0.000706,-0.000435,-0.000708,-0.000683,-0.000259,-0.000676,0.025112,0.001687,-0.000706,...,-0.000708,-0.000706,-0.000526,-0.000709,-0.000706,0.020235,-0.000676,-0.000206,-0.000257,-0.000601


### 사용자 맞춤 앱 추천
- 사용자 아이디에 SVD로 나온 결과가 가장 높은 데이터 순으로 정렬
- 사용자가 설치한(리뷰한) 데이터를 제외
- 사용자가 설치하지 않은 앱에서 가중평점이 높은 10개를 추천

- 특정 사용자를 기준으로 앱마다 유사도를 매긴 데이터 추출

In [16]:
author = '"Raggy Stylz" Doggy Fashion'

user_data = df_svd_preds[df_svd_preds.index == author].transpose().sort_values(by=author, ascending=False)
user_data

author,"""Raggy Stylz"" Doggy Fashion"
app_id,
da066a0b-53fc-42a2-b329-e0cce559f7ea,4.919211
f9e45fcc-4d6a-4dbe-8e3f-7f7c22398ee1,3.682788
ceccb5e3-1424-4cf0-b6a9-7443410f39ad,0.639531
d4d7e4ca-d939-4e8a-857c-37dc62db7c22,0.399802
c98ff6fc-7f5f-441f-b4ac-6271f923ad3f,0.357032
...,...
0c29ae5e-01e0-4efb-959f-40d9041db6fa,-0.055475
4667d9ad-15d5-4b85-954f-b17356814d94,-0.068688
409ab65e-fe31-4f22-a10c-9d0bb06cc30f,-0.069078


- 해당 사용자가 기존에 설치했던 앱들을 가져옴

In [17]:
user_used_app = user_rating.groupby(['author']).get_group(author)
user_used_app

,app_id,author,rating
48423,eaeb2221-5651-4be6-ab9f-268a5076c7bf,"""Raggy Stylz"" Doggy Fashion",4
68810,e18eaef7-59fe-4d47-b36d-2456e755d06c,"""Raggy Stylz"" Doggy Fashion",4
274799,a0bfa0e3-c3ee-4ae3-a39c-798e8737f351,"""Raggy Stylz"" Doggy Fashion",4
371301,f9e45fcc-4d6a-4dbe-8e3f-7f7c22398ee1,"""Raggy Stylz"" Doggy Fashion",4
372703,31f2389f-e4cf-4b6d-9597-cf65da302072,"""Raggy Stylz"" Doggy Fashion",5
375778,da066a0b-53fc-42a2-b329-e0cce559f7ea,"""Raggy Stylz"" Doggy Fashion",5


- 사용자가 설치한 적 없는 앱들만 추천
- 앱의 가중평점으로 정렬해서 상위 10개 앱 추천

In [18]:
recommendations = user_data[~user_data.index.isin(user_used_app['app_id'])]

recommendation10 = recommendations.rename(columns = {author: 'Predictions'}).iloc[:10,:]

user_custom_recommend = pd.merge(left=recommendation10, right=app_rating, how='left', on='app_id').sort_values(by='wr_rating', ascending=False)
user_custom_recommend

,app_id,Predictions,wr_rating
7,baaa4664-a1a4-4b2f-a6a4-dfd0f6ae9b34,0.219056,9.989379
8,604be94e-3903-4276-a327-308a8e245f96,0.208503,9.801890
5,92a6db70-575d-442f-a05c-0a132c0621d2,0.249068,8.693511
4,03246394-bf13-4bbd-b220-8f1c77d15416,0.279451,8.620028
6,f2a160b0-6a8e-4e59-b768-665cb7910426,0.219245,8.513781
9,94e77128-9474-4076-b4a1-2ac6b880c8f3,0.174456,8.383982
1,d4d7e4ca-d939-4e8a-857c-37dc62db7c22,0.399802,8.324730
3,35fca70c-79c7-4a53-beba-db6e0a6c145a,0.347701,8.222394
2,c98ff6fc-7f5f-441f-b4ac-6271f923ad3f,0.357032,7.948691
0,ceccb5e3-1424-4cf0-b6a9-7443410f39ad,0.639531,7.937724
